# Lab 2.1 – RAG with Watsonx, Chroma, and LangChain (Google Colab Version)

![watsonx](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)

---

## 🚀 Run this notebook in Google Colab

**Prerequisites:**
- IBM Cloud API Key ([Create one here](https://cloud.ibm.com/iam/apikeys))
- Watsonx Project ID ([Find it in your watsonx.ai project](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp))

This notebook demonstrates **Retrieval Augmented Generation (RAG)** using:
- **Watsonx.ai** for LLM inference (Granite models)
- **Chroma** as vector database
- **LangChain** for RAG orchestration

---

## Step 1: Install Dependencies

First, we install all required libraries for this RAG implementation.

In [ ]:
# Install all required dependencies
!pip install -qU wget
!pip install -qU "langchain>=0.3,<0.4"
!pip install -qU "ibm_watsonx_ai>=1.1.22"
!pip install -qU "langchain_ibm>=0.3,<0.4"
!pip install -qU "langchain_chroma>=0.1,<0.2"

print("✅ All dependencies installed successfully!")

## Step 2: Configure Credentials

Enter your IBM Cloud API Key and Watsonx Project ID when prompted.

In [ ]:
import os
import getpass

# Prompt for Watsonx credentials
watsonx_api_key = getpass.getpass("Enter IBM Cloud API Key: ")
project_id = getpass.getpass("Enter Watsonx Project ID: ")
url = "https://us-south.ml.cloud.ibm.com"  # Default region

# Store in environment variables
os.environ["WATSONX_APIKEY"] = watsonx_api_key
os.environ["PROJECT_ID"] = project_id
os.environ["WATSONX_URL"] = url

print("✅ Credentials configured successfully!")

## Step 3: Initialize Watsonx API Client

In [ ]:
from ibm_watsonx_ai import Credentials, APIClient

credentials = Credentials(
    url=url,
    api_key=watsonx_api_key,
)

api_client = APIClient(credentials=credentials, project_id=project_id)

print("✅ Watsonx API Client initialized!")

## Step 4: Download Sample Data

We'll use the State of the Union speech as our knowledge base.

In [ ]:
import wget

filename = 'state_of_the_union.txt'
url_data = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/foundation_models/state_of_the_union.txt'

if not os.path.isfile(filename):
    wget.download(url_data, out=filename)
    print(f"\n✅ Downloaded {filename}")
else:
    print(f"✅ {filename} already exists")

## Step 5: Build Knowledge Base with Chroma

We'll:
1. Load the document
2. Split it into chunks
3. Create embeddings using Watsonx
4. Store in Chroma vector database

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma

# Load document
loader = TextLoader(filename)
documents = loader.load()

# Split into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(f"✅ Split document into {len(texts)} chunks")

## Step 6: Create Embeddings with Watsonx

View available embedding models:

In [ ]:
# Show available embedding models
api_client.foundation_models.EmbeddingModels.show()

In [ ]:
from langchain_ibm import WatsonxEmbeddings

embeddings = WatsonxEmbeddings(
    model_id="ibm/slate-30m-english-rtrvr",
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id
)

# Create Chroma vector store
docsearch = Chroma.from_documents(texts, embeddings)

print("✅ Vector database created with Chroma!")

## Step 7: Initialize Watsonx LLM (Granite Model)

In [ ]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import WatsonxLLM

model_id = ModelTypes.GRANITE_13B_CHAT_V2

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

watsonx_granite = WatsonxLLM(
    model_id=model_id.value,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

print("✅ Watsonx Granite LLM initialized!")

## Step 8: Create RAG Chain

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=watsonx_granite,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

print("✅ RAG chain created!")

## Step 9: Test the RAG System

In [ ]:
query = "What did the president say about Ketanji Brown Jackson?"
result = qa.invoke(query)

print("\n" + "="*80)
print(f"Question: {result['query']}")
print("="*80)
print(f"Answer: {result['result']}")
print("="*80)

## Step 10: Try Your Own Questions!

In [ ]:
# Try asking different questions about the State of the Union
your_question = input("Enter your question: ")
result = qa.invoke(your_question)

print("\n" + "="*80)
print(f"Question: {result['query']}")
print("="*80)
print(f"Answer: {result['result']}")
print("="*80)

---

## Summary

You successfully completed this notebook! You learned how to:

✅ Set up Watsonx.ai credentials in Google Colab  
✅ Create embeddings using Watsonx embedding models  
✅ Build a vector database with Chroma  
✅ Implement RAG using LangChain and Watsonx Granite models  

For more information:
- [Watsonx.ai Documentation](https://ibm.github.io/watsonx-ai-python-sdk/samples.html)
- [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction)

---

**Copyright © 2023, 2024 IBM. This notebook and its source code are released under the terms of the MIT License.**